In [1]:
import os
import glob
import pandas as pd
import numpy as np
import ast

In [2]:
file1 = "./summary_alphaPbO2_deltaHvsigma_ordered_Ei_minbin.csv"
df_aPbO2_comp = pd.read_csv(file1)

file2 = "./summary_4comp_bad_Ei_order_binmin.csv"
df_bad_comp = pd.read_csv(file2)

file3 = "./summary_4comp_rutile_Ei_order_binmin.csv"
df_rut_comp = pd.read_csv(file3)

In [3]:
df1 = df_aPbO2_comp
df2 = df_bad_comp
df4 = df_rut_comp

# Function to preprocess each DataFrame by appending titles to column names
def preprocess_df_with_titles(df, title):
    df = df[['Formula', 'sigma bond length', 'Cation sigma', 'DeltaH']]
    df.columns = ['Formula'] + [f"{col}_{title}" for col in df.columns[1:]]
    return df

def normalize_formula(formula):
    # Convert string representation to a Python list
    elements = ast.literal_eval(formula)
    # Sort elements and convert back to string
    return str(sorted(elements))

# Normalize the formulas in both DataFrames
df1['Formula'] = df1['Formula'].apply(normalize_formula)
df2['Formula'] = df2['Formula'].apply(normalize_formula)
df4['Formula'] = df4['Formula'].apply(normalize_formula)

titles = ['aPbO2', 'bad', 'fluor', 'rut']
# Preprocess each DataFrame with respective titles
df1 = preprocess_df_with_titles(df1, titles[0])
df2 = preprocess_df_with_titles(df2, titles[1])
df4 = preprocess_df_with_titles(df4, titles[3])

# Merge all DataFrames on 'Formula' with an outer join
merged_df = df1.merge(df2, on='Formula', how='outer') \
               .merge(df4, on='Formula', how='outer')

# Display the final merged DataFrame
merged_df.head()

,Formula,sigma bond length_aPbO2,Cation sigma_aPbO2,DeltaH_aPbO2,sigma bond length_bad,Cation sigma_bad,DeltaH_bad,sigma bond length_rut,Cation sigma_rut,DeltaH_rut
0,"['Ce', 'Ge', 'Hf', 'Ir']",0.127892,0.072767,0.461590,0.130436,0.075256,0.466841,0.120833,0.088627,0.457033
1,"['Ce', 'Ge', 'Hf', 'Mn']",0.146676,0.084239,0.325290,0.141323,0.084368,0.323455,0.130369,0.092075,0.354483
2,"['Ce', 'Ge', 'Hf', 'Nb']",0.163084,0.094952,0.518559,NaN,NaN,NaN,0.152112,0.107321,0.578145
3,"['Ce', 'Ge', 'Hf', 'Pb']",0.143940,0.069778,0.291689,0.146666,0.072580,0.297499,0.133715,0.085034,0.342001
4,"['Ce', 'Ge', 'Hf', 'Pt']",0.128346,0.074133,0.354258,0.125677,0.081444,0.354506,0.116889,0.084023,0.360637


In [4]:
titles = ['aPbO2','bad','rut']
wsigma = 0.0299359345584294*np.sqrt(2)
wH = 0.0750966429584929*np.sqrt(2)

for crys in titles:
    colname = "rho_"+crys
    merged_df[colname] = merged_df.apply(lambda row: np.sqrt( (row["Cation sigma_"+crys]/wsigma)**2 + (row["DeltaH_"+crys]/wH)**2),axis=1)
    

In [5]:
deltaH_cols = [col for col in merged_df.columns if col.startswith("rho_")]
merged_df['Min_rho'] = merged_df[deltaH_cols].min(axis=1)
merged_df['Min_Crystal_rho'] = merged_df[deltaH_cols].idxmin(axis=1).str.split('_').str[-1]
merged_df = merged_df.sort_values(by='Min_rho')

In [6]:
merged_df.head(20)

,Formula,sigma bond length_aPbO2,Cation sigma_aPbO2,DeltaH_aPbO2,sigma bond length_bad,Cation sigma_bad,DeltaH_bad,sigma bond length_rut,Cation sigma_rut,DeltaH_rut,rho_aPbO2,rho_bad,rho_rut,Min_rho,Min_Crystal_rho
628,"['Hf', 'Pb', 'Sn', 'Zr']",0.065397,0.032896,0.057140,0.066538,0.036035,0.057542,0.041580,0.033870,0.047034,0.945108,1.008977,0.914431,0.914431,rut
664,"['Hf', 'Sn', 'Ti', 'Zr']",0.094455,0.029936,0.075097,NaN,NaN,NaN,0.066255,0.032920,0.079332,1.000000,NaN,1.078258,1.000000,aPbO2
751,"['Ir', 'Pt', 'Rh', 'Ru']",NaN,NaN,NaN,NaN,NaN,NaN,0.148573,0.039978,0.037992,NaN,NaN,1.009792,1.009792,rut
626,"['Hf', 'Pb', 'Sn', 'Ti']",0.098810,0.035875,0.101784,0.100108,0.035562,0.103064,0.077415,0.033567,0.078150,1.279286,1.283504,1.081725,1.081725,rut
613,"['Hf', 'Pb', 'Pt', 'Sn']",NaN,NaN,NaN,0.070457,0.049715,0.113711,0.060433,0.038467,0.085627,NaN,1.589145,1.214754,1.214754,rut
641,"['Hf', 'Pt', 'Sn', 'Ti']",0.066732,0.043735,0.118722,0.066938,0.046483,0.109328,0.050232,0.042128,0.074849,1.522117,1.505077,1.219387,1.219387,rut
958,"['Pb', 'Sn', 'Ti', 'Zr']",0.102560,0.037092,0.096296,0.105499,0.037213,0.090494,0.085680,0.041022,0.085259,1.260865,1.224218,1.258317,1.224218,bad
630,"['Hf', 'Pb', 'Ti', 'Zr']",0.109435,0.036594,0.106772,0.106625,0.035773,0.108955,0.086395,0.033514,0.103019,1.325856,1.329093,1.252039,1.252039,rut
415,"['Ge', 'Mn', 'Ti', 'V']",NaN,NaN,NaN,0.086187,0.060839,0.116704,0.047214,0.048684,0.054148,NaN,1.809051,1.257908,1.257908,rut
331,"['Ge', 'Hf', 'Sn', 'Ti']",0.096572,0.038340,0.127128,0.094645,0.039369,0.130397,0.070286,0.037126,0.100896,1.501008,1.540225,1.292900,1.292900,rut


In [7]:
deltaH_cols = [col for col in merged_df.columns if col.startswith("DeltaH_")]
# merged_df['Min_DeltaH'] = merged_df[deltaH_cols].min(axis=1)
merged_df['Min_Crystal_DeltaH'] = merged_df[deltaH_cols].idxmin(axis=1).str.split('_').str[-1]

In [8]:
merged_df

,Formula,sigma bond length_aPbO2,Cation sigma_aPbO2,DeltaH_aPbO2,sigma bond length_bad,Cation sigma_bad,DeltaH_bad,sigma bond length_rut,Cation sigma_rut,DeltaH_rut,rho_aPbO2,rho_bad,rho_rut,Min_rho,Min_Crystal_rho,Min_Crystal_DeltaH
628,"['Hf', 'Pb', 'Sn', 'Zr']",0.065397,0.032896,0.057140,0.066538,0.036035,0.057542,0.041580,0.033870,0.047034,0.945108,1.008977,0.914431,0.914431,rut,rut
664,"['Hf', 'Sn', 'Ti', 'Zr']",0.094455,0.029936,0.075097,NaN,NaN,NaN,0.066255,0.032920,0.079332,1.000000,NaN,1.078258,1.000000,aPbO2,aPbO2
751,"['Ir', 'Pt', 'Rh', 'Ru']",NaN,NaN,NaN,NaN,NaN,NaN,0.148573,0.039978,0.037992,NaN,NaN,1.009792,1.009792,rut,rut
626,"['Hf', 'Pb', 'Sn', 'Ti']",0.098810,0.035875,0.101784,0.100108,0.035562,0.103064,0.077415,0.033567,0.078150,1.279286,1.283504,1.081725,1.081725,rut,rut
613,"['Hf', 'Pb', 'Pt', 'Sn']",NaN,NaN,NaN,0.070457,0.049715,0.113711,0.060433,0.038467,0.085627,NaN,1.589145,1.214754,1.214754,rut,rut
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,"['Ce', 'Hf', 'Ir', 'Nb']",0.112632,0.084581,0.603963,0.131638,0.089863,0.580912,0.104667,0.086277,0.596077,6.027610,5.867256,5.971160,5.867256,bad,bad
134,"['Ce', 'Ir', 'Nb', 'Ti']",0.155463,0.085225,0.602992,0.158270,0.074933,0.595066,0.130600,0.086657,0.605865,6.024057,5.876028,6.060900,5.876028,bad,bad
132,"['Ce', 'Ir', 'Nb', 'Ru']",NaN,NaN,NaN,0.115502,0.084847,0.634267,0.111272,0.076903,0.647576,NaN,6.299539,6.362372,6.299539,bad,bad
32,"['Ce', 'Ge', 'Nb', 'Ru']",0.149143,0.100796,0.622408,NaN,NaN,NaN,0.159165,0.104645,0.641258,6.325723,NaN,6.524402,6.325723,aPbO2,aPbO2


In [9]:
cat_cols = [col for col in merged_df.columns if col.startswith("Cation sigma_")]
# merged_df['Min_sigma'] = merged_df[cat_cols].min(axis=1)
merged_df['Min_Crystal_sigma'] = merged_df[cat_cols].idxmin(axis=1).str.split('_').str[-1]
# merged_df = merged_df.sort_values(by='Min_sigma')
merged_df

,Formula,sigma bond length_aPbO2,Cation sigma_aPbO2,DeltaH_aPbO2,sigma bond length_bad,Cation sigma_bad,DeltaH_bad,sigma bond length_rut,Cation sigma_rut,DeltaH_rut,rho_aPbO2,rho_bad,rho_rut,Min_rho,Min_Crystal_rho,Min_Crystal_DeltaH,Min_Crystal_sigma
628,"['Hf', 'Pb', 'Sn', 'Zr']",0.065397,0.032896,0.057140,0.066538,0.036035,0.057542,0.041580,0.033870,0.047034,0.945108,1.008977,0.914431,0.914431,rut,rut,aPbO2
664,"['Hf', 'Sn', 'Ti', 'Zr']",0.094455,0.029936,0.075097,NaN,NaN,NaN,0.066255,0.032920,0.079332,1.000000,NaN,1.078258,1.000000,aPbO2,aPbO2,aPbO2
751,"['Ir', 'Pt', 'Rh', 'Ru']",NaN,NaN,NaN,NaN,NaN,NaN,0.148573,0.039978,0.037992,NaN,NaN,1.009792,1.009792,rut,rut,rut
626,"['Hf', 'Pb', 'Sn', 'Ti']",0.098810,0.035875,0.101784,0.100108,0.035562,0.103064,0.077415,0.033567,0.078150,1.279286,1.283504,1.081725,1.081725,rut,rut,rut
613,"['Hf', 'Pb', 'Pt', 'Sn']",NaN,NaN,NaN,0.070457,0.049715,0.113711,0.060433,0.038467,0.085627,NaN,1.589145,1.214754,1.214754,rut,rut,rut
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,"['Ce', 'Hf', 'Ir', 'Nb']",0.112632,0.084581,0.603963,0.131638,0.089863,0.580912,0.104667,0.086277,0.596077,6.027610,5.867256,5.971160,5.867256,bad,bad,aPbO2
134,"['Ce', 'Ir', 'Nb', 'Ti']",0.155463,0.085225,0.602992,0.158270,0.074933,0.595066,0.130600,0.086657,0.605865,6.024057,5.876028,6.060900,5.876028,bad,bad,bad
132,"['Ce', 'Ir', 'Nb', 'Ru']",NaN,NaN,NaN,0.115502,0.084847,0.634267,0.111272,0.076903,0.647576,NaN,6.299539,6.362372,6.299539,bad,bad,rut
32,"['Ce', 'Ge', 'Nb', 'Ru']",0.149143,0.100796,0.622408,NaN,NaN,NaN,0.159165,0.104645,0.641258,6.325723,NaN,6.524402,6.325723,aPbO2,aPbO2,aPbO2


In [10]:
csvfilestr = "./SUMMARY_4comp_alpha_bad_rutile_rho_ordered.csv"
merged_df.to_csv(csvfilestr, index=False)

In [ ]:
exclude_list = ['']
filtered_df = merged_df.copy()

for element in exclude_list:
    filtered_df = filtered_df[~filtered_df['Formula'].str.contains(fr"{element}", na=False)]

filtered_df